In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at020-state-machine"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='3d30c728-2634-4ec2-a132-dc0ac143e484', created_at=datetime.datetime(2025, 7, 28, 4, 3, 11, 172316, tzinfo=TzInfo(UTC)), name='4759e77b-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 28, 4, 3, 11, 172316, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='14a48389-0c6e-4001-9c11-8e4e6edf9390', agent_id='8e8d7d03-214f-4163-97b4-ce687ce9923f', sequence_id=253, task_id='3d30c728-2634-4ec2-a132-dc0ac143e484', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 4, 3, 11, 382344, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [07/28/2025 04:03:11] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:03:11] ─────────────────────────╮
│ What specific areas of AI or types of startups are you most interested in?   │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 20 seconds - returning collected messages


In [6]:
# Send a follow up  event to the agent in response to the agent's follow up question

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "I want to know what viral news came up and which startups failed, got acquired, or became very successful or popular in the last 3 months"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='d52b0a3e-bb39-4898-a313-d342cfe29dee', agent_id='8e8d7d03-214f-4163-97b4-ce687ce9923f', sequence_id=254, task_id='3d30c728-2634-4ec2-a132-dc0ac143e484', content=TextContent(author='user', content='I want to know what viral news came up and which startups failed, got acquired, or became very successful or popular in the last 3 months', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 4, 4, 58, 62398, tzinfo=TzInfo(UTC)))


In [7]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=30, # Notice the longer timeout to give time for the agent to respond
)

╭───────────────────────── USER [07/28/2025 04:04:58] ─────────────────────────╮
│ I want to know what viral news came up and which startups failed, got        │
│ acquired, or became very successful or popular in the last 3 months          │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:04:58] ─────────────────────────╮
│ Starting deep research process based on your query and follow-up             │
│ responses...                                                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:05:13] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest news AI startups October 2023",                          │
│    "model": "gpt-4o",                                                        │
│    "type": "web_search_preview"                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:05:13] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "In October 2023, the AI startup landscape witnessed significant  │
│  developments, including substantial funding rounds, strategic partnerships  │
│  and innovative product launches. Here are some of the notable               │
│  highlights:\n\n**Major Funding Rounds:**\n\n- **Mistral AI:** The           │
│  Paris-based generative AI startup, co-founded by former researchers from    │
│  Meta Platforms and Alphabet, raised \u20ac385 million in October 2023. Thi  │
│  funding aimed to position Mistral AI as Europe's counterpart to OpenAI.     │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Mistral_AI?utm_source=op  │
│  ai))\n\n- **Perplexity:** Specializing in conversational AI search engines  │
│  Perplexity secured new funding led by venture capital firm IVP, elevating   │
│  its valuation to $500 million.                                              │
│  ([techstartups.com](https://techstartups.com/2023/10/24/?utm_source=openai  │
│  \n\n- **EdgeCortix:** The Tokyo-based AI hardware company raised $20 milli  │
│  in a funding round led by SBI Investment and Global Hands-On VC. EdgeCorti  │
│  focuses on scalable, run-time reconfigurable neural network processor IP f  │
│  edge inference.                                                             │
│  ([semiengineering.com](https://semiengineering.com/startup-funding-october  │
│  023/?utm_source=openai))\n\n- **Lemurian Labs:** This Toronto-based startu  │
│  developing hardware and software accelerated computing platforms for AI     │
│  applications secured $9 million in seed funding.                            │
│  ([semiengineering.com](https://semiengineering.com/startup-funding-october  │
│  023/?utm_source=openai))\n\n**Strategic Partnerships and                    │
│  Acquisitions:**\n\n- **Flatfile's Acquisition of ChatCSV:** Denver-based    │
│  Flatfile acquired ChatCSV, an AI startup known for its AI chat app used by  │
│  organizations like P&G and McKinsey. This acquisition aims to enhance       │
│  Flatfile's AI-powered data exchange platform.                               │
│  ([techstartups.com](https://techstartups.com/2023/10/17/?utm_source=openai  │
│  \n\n- **WasteVision AI and Keymakr Collaboration:** Announced on October 6  │
│  2023, WasteVision AI partnered with Keymakr to revolutionize waste          │
│  management through technological advancements, focusing on innovative       │
│  solutions for waste sorting and detection.                                  │
│  ([hackernoon.com](https://hackernoon.com/jasper-lowers-valuation-and-other  │
│  i-news-during-september-and-october?utm_source=openai))\n\n**Product        │
│  Launches and Innovations:**\n\n- **Qwak's Vector Store:** On September 28,  │
│  2023, Israeli MLOps startup Qwak unveiled its Vector Store, a feature       │
│  designed to transform how data science teams manage vector storage,         │
│  retrieval, and embedding operations, streamlining these processes.          │
│  ([hackernoon.com](https://hackernoon.com/jasper-lowers-valuation-and-other  │
│  i-news-during-september-and-october?utm_source=openai))\n\n- **Giga ML's    │
│  On-Premise LLMs:** Bengaluru-based Giga ML introduced on-premise deploymen  │
│  and fine-tuning services for large language models (LLMs), allowing         │
│  enterprises to utilize LLMs i

╭──────────────────────── AGENT [07/28/2025 04:05:22] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "AI startup acquisitions October 2023",                          │
│    "model": "gpt-4o",                                                        │
│    "type": "web_search_preview"                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────── AGENT [07/28/2025 04:05:29] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "AI startup failures October 2023",                              │
│    "model": "gpt-4o",                                                        │
│    "type": "web_search_preview"                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:05:29] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "In October 2023, several AI startups faced significant challeng  │
│  leading to their closures. Notable examples include:\n\n**Olive AI**:       │
│  Founded in 2012, Olive AI aimed to revolutionize healthcare through         │
│  automation. Despite raising approximately $902 million and achieving a      │
│  valuation of $4 billion by 2021, the company struggled with rapid growth a  │
│  strategic missteps. These issues led to multiple layoffs and, ultimately,   │
│  the cessation of operations in October 2023.                                │
│  ([aimresearch.co](https://aimresearch.co/market-industry/big-players-misst  │
│  s-of-turning-billion-dollar-startups-to-failures?utm_source=openai))\n\n**  │
│  tifact**: Launched in February 2023 by Instagram co-founders Kevin Systrom  │
│  and Mike Krieger, Artifact was an AI-driven news app designed to personali  │
│  news curation. Despite an initial surge of 100,000 downloads, user          │
│  engagement declined, with only 12,000 new installs by October 2023. The     │
│  app's expansion into additional features diluted its core value propositio  │
│  leading to its shutdown.                                                    │
│  ([newsletter.failory.com](https://newsletter.failory.com/p/2024-s-biggest-  │
│  artup-crashes?utm_source=openai))\n\nThese cases highlight the challenges   │
│  startups can face, including rapid scaling, market demand misalignment, an  │
│  the complexities of integrating AI into practical applications. ",          │
│    "annotations": null,                                                      │
│    "meta": null                                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:05:41] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "successful AI startups October 2023",                           │
│    "model": "gpt-4o",                                                        │
│    "type": "web_search_preview"                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:05:41] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "In October 2023, several AI startups achieved significant        │
│  milestones, reflecting the dynamic growth in the artificial intelligence    │
│  sector. Notable examples include:\n\n**Mistral AI**: A French AI startup    │
│  specializing in open-weight large language models. In October 2023, Mistra  │
│  AI raised \u20ac385 million ($428 million) in a funding round led by        │
│  Andreessen Horowitz, BNP Paribas, and Salesforce, valuing the company at    │
│  over $2 billion.                                                            │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Mistral_AI?utm_source=op  │
│  ai))\n\n**Unitary**: A UK-based company focused on AI-driven content        │
│  moderation. In October 2023, Unitary secured a $15 million Series A fundin  │
│  round led by Creandum, with participation from Paladin Capital and Plural.  │
│  Their AI system processes six million videos daily, enhancing online safet  │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Unitary_%28company%29?ut  │
│  source=openai))\n\n**Neurons**: Based in Denmark, Neurons provides AI       │
│  solutions to optimize advertisements and increase conversions. By October   │
│  2023, the company had an annual revenue of \u20ac4 million and plans to     │
│  secure \u20ac10 million in funding, with \u20ac5 million already committed  │
│  Investors include Fairpoint Capital and EIFO.                               │
│  ([vestbee.com](https://www.vestbee.com/blog/articles/startups-of-the-month  │
│  ctober-2023?utm_source=openai))\n\n**Neysa**: An Indian startup offering a  │
│  cloud platform for AI acceleration and high-performance computing           │
│  infrastructure. In October 2023, Neysa raised $30 million in a funding rou  │
│  led by NTTVC, Z47, and Nexus Venture Partners, bringing its total funding   │
│  $50 million.                                                                │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Neysa?utm_source=openai)  │
│  n\n**Imbue**: Formerly known as Generally Intelligent, Imbue focuses on     │
│  developing AI systems with reasoning capabilities. In October 2023, the     │
│  company secured an additional $12 million from Amazon\u2019s Alexa Fund an  │
│  former Google CEO Eric Schmidt, following a $200 million Series B funding   │
│  September.                                                                  │
│  ([iguides.org](https://www.iguides.org/top-ai-startups/?utm_source=openai)  │
│  n\n**Cohere**: Specializing in large language models for enterprise AI,     │
│  Cohere's technology is embedded into products from companies like Oracle a  │
│  Salesforce. In 2023, Cohere was valued at about $2 billion following a      │
│  funding round.                                                              │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Cohere?utm_source=openai  │
│  \n\nThese developments underscore the rapid advancement and investment in   │
│  technologies during that period. ",                                         │
│    "annotations": null,                                                      │
│    "meta": null                                                              │
│  }                                                                           │
│                               

KeyboardInterrupt: 